In [1]:
import findspark
findspark.init()

import pyspark
import random
import os
import glob
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
#init spark context and spark session

# http://www.portaldatransparencia.gov.br/download-de-dados/viagens
#
# Viagens realizadas a servico
#
# CONTROLADORIA-GERAL DA UNIÃO
#


sc = pyspark.SparkContext(appName="projetoViagens")
spark = SparkSession.builder.appName('projetoViagens').getOrCreate()

sc

<SparkContext master=local[*] appName=projetoViagens>

In [3]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['2019_Passagem_.csv',
 '2019_Trecho_.csv',
 '2019_Viagem_.csv',
 '2019_Viagem.csv',
 '2019_Pagamento_.csv',
 '2019_Trecho.csv',
 '2019_Pagamento.csv',
 '2019_Passagem.csv']

In [4]:
#using pandas
#read all csv files
#csv_files = [pd.read_csv(f, sep='";"').apply(lambda x: x.astype(str).str.lower()) for f in all_filenames]

#for csv in csv_files:
    #lowercase all columns
#    csv.columns = [x.lower() for x in csv.columns]

#combine all files in the list
#combined_csv = pd.concat(csv_files, sort=False)

#export to csv
#combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig', sep=';')

#combined_csv

In [4]:
#colunas
#"codigov";"situacao";"codigo_orgao_superior";"nome_orgao_superior";"codigo_orgao_solicitantev";"nome_orgao_solicitante";"cpf_viajante";"nome";"cargo";"periodo_data_inicio";"periodo_data_fim";"destinos";"motivo";"valor_diarias";"valor_passagens";"valor_outros_gastos"
#read viagem csv
file_viagem = ['2019_Viagem_.csv']
#file_viagem = ['2019_Viagem.csv']
csv_viagem = [pd.read_csv(f, sep=';').apply(lambda x: x.astype(str).str.lower().str.replace(',','.')) for f in file_viagem]

for csv in csv_viagem:
    #lowercase all columns
    csv.columns = [x.lower() for x in csv.columns]

#combine all files in the list
combined_csv_viagem = pd.concat(csv_viagem, sort=False)


Tableviagemcsv = spark.createDataFrame(combined_csv_viagem,[
    'codigov','situacao','codigo_orgao_superior','nome_orgao_superior','codigo_orgao_solicitantev',
    'nome_orgao_solicitante','cpf_viajante','nome','cargo','periodo_data_inicio',
    'periodo_data_fim','destinos','motivo','valor_diarias','valor_passagens','valor_outros_gastos'])

Tableviagemcsv.write.parquet("Tableviagem9")

Tableviagem = spark.read.parquet("Tableviagem9")
Tableviagem.show()

+--------+-------------+---------------------+--------------------+-------------------------+----------------------+--------------+--------------------+--------------------+-------------------+----------------+--------------------+--------------------+-------------+---------------+-------------------+
| codigov|     situacao|codigo_orgao_superior| nome_orgao_superior|codigo_orgao_solicitantev|nome_orgao_solicitante|  cpf_viajante|                nome|               cargo|periodo_data_inicio|periodo_data_fim|            destinos|              motivo|valor_diarias|valor_passagens|valor_outros_gastos|
+--------+-------------+---------------------+--------------------+-------------------------+----------------------+--------------+--------------------+--------------------+-------------------+----------------+--------------------+--------------------+-------------+---------------+-------------------+
|15163874|    realizada|                26000|ministério da edu...|                    2629

In [5]:
#colunas
#"codigot";"sequencia_trecho";"origem_data";"origem_pais";"origem_uf";"origem_cidade";"destino_data";"destino_pais";"destino_uf";"destino_cidade";"meio_transporte";"numero_diarias";"missao"
#read trecho csv
file_trecho = ['2019_Trecho_.csv']
#file_trecho = ['2019_Trecho.csv']
csv_trecho = [pd.read_csv(f, sep=';').apply(lambda x: x.astype(str).str.lower().str.replace(',','.')) for f in file_trecho]

for csv in csv_trecho:
    #lowercase all columns
    csv.columns = [x.lower() for x in csv.columns]

#combine all files in the list
combined_csv_trecho = pd.concat(csv_trecho, sort=False)

#valuesA = [('Pirate',1),('Monkey',2),('Ninja',3),('Spaghetti',4)]
Tabletrecho = spark.createDataFrame(combined_csv_trecho,[
    'codigot','sequencia_trecho','origem_data','origem_pais','origem_uf','origem_cidade','destino_data',
    'destino_pais','destino_uf','destino_cidade','meio_transporte','numero_diarias','missao'])
Tabletrecho.show(5)

+--------+----------------+-----------+--------------------+----------------+--------------------+------------+------------+-----------------+--------------+---------------+--------------+------+
| codigot|sequencia_trecho|origem_data|         origem_pais|       origem_uf|       origem_cidade|destino_data|destino_pais|       destino_uf|destino_cidade|meio_transporte|numero_diarias|missao|
+--------+----------------+-----------+--------------------+----------------+--------------------+------------+------------+-----------------+--------------+---------------+--------------+------+
|15163874|               1|   20190217|estados unidos da...|             nan|          washington|    20190218|      brasil|            bahia|      salvador|          aéreo|           .00|   sim|
|15166192|               1|   20190220|              itália|             nan|                roma|    20190221|      brasil|   rio de janeiro|rio de janeiro|          aéreo|           .00|   sim|
|15214826|          

In [7]:
#colunas
#"codigop";"meio_transporte";"pais_origem_ida";"uf_origem_ida";"cidade_origem_ida";"pais_destino_ida";"uf_destino_ida";"cidade_destino_ida";"pais_origem_volta";"uf_origem_volta";"cidade_origem_volta";"pais_destino_volta";"uf_destino_volta";"cidade_destino_volta";"valor_passagem";"taxa_servico"
#read passagem csv
#file_passagem = ['2019_Passagem_.csv']
#file_passagem = ['2019_Passagem.csv']
#csv_passagem = [pd.read_csv(f, sep=';').apply(lambda x: x.astype(str).str.lower().str.replace(',','.')) for f in file_passagem]

#for csv in csv_passagem:
    #lowercase all columns
#    csv.columns = [x.lower() for x in csv.columns]

#combine all files in the list
#combined_csv_passagem = pd.concat(csv_passagem, sort=False)

#valuesA = [('Pirate',1),('Monkey',2),('Ninja',3),('Spaghetti',4)]
#Tablepassagem = spark.createDataFrame(combined_csv_passagem,[
#    'codigop','meio_transporte','pais_origem_ida','uf_origem_ida','cidade_origem_ida','pais_destino_ida',
#    'uf_destino_ida','cidade_destino_ida','pais_origem_volta','uf_origem_volta','cidade_origem_volta',
#    'pais_destino_volta','uf_destino_volta','cidade_destino_volta','valor_passagem','taxa_servico'])
#Tablepassagem.show()

In [28]:
#colunas
#"codigopg";"codigo_orgao_superior";"nome_orgao_superior";"codigo_orgao_pagador";"nome_orgao_pagador";"codigo_unidade_gestora_pagadora";"nome_unidade_gestora_pagadora";"tipo_pagamento";"valor"
#read pagamento csv
#file_pagamento = ['2019_Pagamento_.csv']
#file_pagamento = ['2019_Pagamento.csv']
#csv_pagamento = [pd.read_csv(f, sep=';').apply(lambda x: x.astype(str).str.lower().str.replace(',','.')) for f in file_pagamento]

#for csv in csv_pagamento:
    #lowercase all columns
#    csv.columns = [x.lower() for x in csv.columns]

#combine all files in the list
#combined_csv_pagamento = pd.concat(csv_pagamento, sort=False)

#valuesA = [('Pirate',1),('Monkey',2),('Ninja',3),('Spaghetti',4)]
#Tablepagamento = spark.createDataFrame(combined_csv_pagamento,[
#    'codigopg','codigo_orgao_superior','nome_orgao_superior','codigo_orgao_pagador','nome_orgao_pagador',
#    'codigo_unidade_gestora_pagadora','nome_unidade_gestora_pagadora','tipo_pagamento','valor'])
#Tablepagamento.show()

In [6]:
#retorno_consulta = Tableviagem.join(Tabletrecho, Tableviagem.codigov == Tabletrecho.codigot).join(Tablepassagem, Tablepassagem.codigop == Tableviagem.codigov).join(Tablepagamento, Tablepagamento.codigopg == Tablepassagem.codigop)
retorno_consulta = Tableviagem.join(Tabletrecho, Tableviagem.codigov == Tabletrecho.codigot)
retorno_consulta_filter = retorno_consulta.filter("situacao = 'realizada'")
#retorno_consulta_filter = Tableviagem.filter("situacao = 'realizada' and periodo_data_fim < '20190131'")

#retorno_consulta_filter = Tableviagem

In [7]:
#quantidade de viagens em 2019
print('Quantidade de viagens em 2019: ', retorno_consulta_filter.select('codigov').distinct().count())

Quantidade de viagens em 2019:  24


In [8]:
#gasto total com viagens em 2019
print('Gasto total com viagens: ')
(retorno_consulta_filter.select('codigov', 'valor_passagens').distinct()).agg({'valor_passagens': 'sum'}).show()


Gasto total com viagens: 
+--------------------+
|sum(valor_passagens)|
+--------------------+
|  40422.990000000005|
+--------------------+



In [9]:
# orgao que mais gastou com viagens
retorno_consulta_filter.groupby(['codigo_orgao_solicitantev', 'nome_orgao_solicitante']).agg({'valor_passagens': 'sum'}).sort(['sum(valor_passagens)'], ascending=[0]).show()

+-------------------------+----------------------+--------------------+
|codigo_orgao_solicitantev|nome_orgao_solicitante|sum(valor_passagens)|
+-------------------------+----------------------+--------------------+
|                    26271|  fundação universi...|            23556.68|
|                    39251|  agência nacional ...|            16803.32|
|                    26291|  fundação coordena...|             7990.98|
|                    26251|  fundação universi...|             7102.56|
|                    26258|  universidade tecn...|              5916.9|
|                    35000|  ministério das re...|             5895.37|
|                    26234|  universidade fede...|              5343.3|
|                    26249|  universidade fede...|             4421.36|
|                    52121|   comando do exército|  4079.5200000000004|
|                    26244|  universidade fede...|              2395.9|
|                    26242|  universidade fede...|             1

In [10]:
# Destinos mais viajados
retorno_consulta_filter.groupby(['destino_pais', 'destino_uf','destino_cidade']).agg({'destino_cidade': 'count'}).sort(['count(destino_cidade)'], ascending=[0]).show()

+------------+-----------------+--------------+---------------------+
|destino_pais|       destino_uf|destino_cidade|count(destino_cidade)|
+------------+-----------------+--------------+---------------------+
|      brasil| distrito federal|      brasília|                   11|
|      brasil|       pernambuco|        recife|                    5|
|      brasil|rio grande do sul|  porto alegre|                    4|
|      brasil|   rio de janeiro|rio de janeiro|                    4|
|      brasil|            bahia|      salvador|                    3|
|      brasil|          alagoas|        maceió|                    3|
|      brasil|        são paulo|     são paulo|                    3|
|      brasil|           paraná|       maringá|                    2|
|      itália|              nan|         milão|                    2|
|      itália|              nan|         pavia|                    2|
|      brasil|            ceará|     fortaleza|                    2|
|      brasil|      